# December 26, 2023 - Notes

So I've been fighting a major lack of Evapotranspiration research as I know the USGS already has a specific dataset dedicated to this.  The problem is the only way to acquire this data is to use their EROS Science Processing Architecture (ESPA).  This is a command-line / API-driven service using a REST interface.  

I finally sat down today and worked out how to use this interface.  It's not as bad as I thought (as usual).  The only quirk is you need to know the datasets you want in advance.  To solve this, I used USGS Earth Explorer to export all available Landsat L2SP datasets to an Excel Spreadsheet.  See notes below on my workflow.

## Ordering Data on USGS ESPA Service

Here is a good set of links to their API:

* [API Readme](https://espa.cr.usgs.gov/static/docs/api-readme.html)
* [API Reference](https://espa.cr.usgs.gov/static/docs/api-resources-list.html)

This service is generaly not helpful unless you are trying to automate imagery collection.  I'd stick to EarthExplorer for normal digging.  In particular, ESPA is the only resource for Evapotranspiration datasets. 

### Step 1:  Verify Account Access

All examples in these notes use `curl`.  Make sure you have it installed. 

Verify you have an account with the USGS EROS service. 

    $> curl --user <USERNAME> -X GET https://espa.cr.usgs.gov/api/v0/user

The response I get back...

```
{"email":"<EMAIL>","first_name":"Marvin","last_name":"Smith","roles":["active"],"username":"<USERNAME>"}
```

### Step 2:  Check Contents of Specific Datasets

This will take a specific Collection-ID and return available datasets associated with it. 

```
$>curl --netrc -X GET -d '{"inputs":["LC08_L1TP_033032_20230517_20230524_02_T1"]}'  https://espa.cr.usgs.gov/api/v0/available-products
```
The response:
```
{"olitirs8_collection_2_l1": {"products": ["l1", "aq_refl", "stats"], "inputs": ["LC08_L1TP_033032_20230517_20230524_02_T1"]}}
```

Here is another:
```
$> curl --netrc -X GET -d '{"inputs":["LC08_L2SP_033032_20230517_20230524_02_T1"]}' https://espa.cr.usgs.gov/api/v0/available-products
```

The response:
```
{"olitirs8_collection_2_l2": {"products": ["l1", "sr_ndvi", "sr_evi", "sr_savi", "sr_msavi", "sr_ndmi", "sr_nbr", "sr_nbr2", "sr_ndsi", "stats", "et"], "inputs": ["LC08_L2SP_033032_20230517_20230524_02_T1"]}}
```

### Step 3:  Check Existing Orders

This is helpful once you are downloading datasets and need to verify if an order is available. 

    $> curl --user <USERNAME> -X GET https://espa.cr.usgs.gov/api/v0/list-orders

My response:
```
Enter host password for user '<USERNAME>':
[]
```

### Step 4: Ordering Evapotranspiration Data

Link about datasets [here](https://www.usgs.gov/landsat-missions/landsat-collection-2-provisional-actual-evapotranspiration-science-product). 

Notes:

* This only works for *L2SP* datasets.
* This currently only works for LC08, not LC09.
* *L2SP* is also (and more importantly) called *Collection 2, Level 2, Science Product.
* In USGS Earth Explorer, this is located in the image below:

![image](../../images/l2sp_ee.png)

I have a convenient export from the USGS site {here}`../../data/landsat_l2sp_usgs_ee_export.xlsx`.

#### Command:

Given a collection-id, use the following command:

```bash
curl --netrc -d '{"olitirs8_collection_2_l2": 
                    { 
                      "inputs": ["LC08_L2SP_033032_20230517_20230524_02_T1"], 
                      "products": ["et"]
                    }, 
                    "format": "gtiff", 
                     "note": "test with API."
                  }' https://espa.cr.usgs.gov/api/v0/order
```
Response:
```
{"orderid":"espa-marvin_smith1@me.com-12262023-161650-926","status":"ordered"}
```

You now have placed an order for Evapotranspiration `(ev)` data from ESPA. 

#### Check the receipt of the request:

Next verify you have the order placed. 

    $> curl --netrc -X GET https://espa.cr.usgs.gov/api/v0/list-orders

Response:
    
    ["espa-marvin_smith1@me.com-12262023-161650-926"]


#### Downloading ESPA Datasets

Install the USGS ESPA Bulk Downloader using PIP with your virtual environment:

    pip install git+https://code.usgs.gov/espa/bulk-downloader.git

**NOTE:** This didn't work for me, so I added it to the repo as a git submodule just to move forward. 

Using my copy:

    #  First time only
    git submodule update --init --recursive
    
    pushd ./utilities/usgs/bulk-downloader

Then run the command:

    ./download_espa_order.py -e marvin_smith1@me.com -o ALL -d ~/Desktop/Imagery/USGS_ESPA -u msmith81886 -v

This is what you should see:

```
Password: 
2023-12-26 15:35:46,740| Retrieving orders: ['espa-marvin_smith1@me.com-12262023-161650-926']
2023-12-26 15:36:02,253| File 1 of 1 for order: espa-marvin_smith1@me.com-12262023-161650-926
2023-12-26 15:36:02,253| Created target_directory: /Users/marvinsmith/Desktop/Imagery/USGS_ESPA/espa-marvin_smith1@me.com-12262023-161650-926 
2023-12-26 15:36:02,253| Downloading LC080330322023051702T1-SC20231226221722, to: /Users/marvinsmith/Desktop/Imagery/USGS_ESPA/espa-marvin_smith1@me.com-12262023-161650-926
```

Looking at the filesystem, we see a tarball ready to unpack. 

    $> ls ~/Desktop/Imagery/USGS_ESPA/espa-marvin_smith1@me.com-12262023-161650-926
    LC080330322023051702T1-SC20231226221722.tar.gz

Unpacking, we see the following contents:

![contents](../../images/usgs_ev_contents.png)

## Misc Stuff

### Fetching all Landsat images from 2020 to 2023 for the month of April. 

I created a simple Python script to make this trivial.  Set the `False` to `True` in order to execute it.

In [1]:
run_script = True
skip_download = True

#  Change this variable to match the ranges
pattern_date = '202[0-3]-05-[0-9]{2} .*'
pattern_lc   = 'LC08_.*'

In [2]:
import pandas as pd
import json, re

if run_script:

    #  Load my special Excel Spreadsheet
    collect_list = pd.read_excel( '../../data/landsat_l2sp_usgs_ee_export.xlsx' )

    print('Sample of collections in file')
    display(collect_list.head(3))
    

Sample of collections in file


,Landsat Product Identifier L2,Landsat Product Identifier L1,Landsat Scene Identifier,Date Acquired,Collection Category,Collection Number,WRS Path,WRS Row,Target WRS Path,Target WRS Row,...,Corner Upper Left Latitude,Corner Upper Left Longitude,Corner Upper Right Latitude,Corner Upper Right Longitude,Corner Lower Left Latitude,Corner Lower Left Longitude,Corner Lower Right Latitude,Corner Lower Right Longitude,Display ID,Entity ID
0,LC08_L2SP_033032_20130318_20200913_02_T1,LC08_L1TP_033032_20130318_20200913_02_T1,LC80330322013077LGN02,2013-03-18,T1,2,33,32,33,32,...,41.32536,-106.22480,41.32266,-103.54346,39.31764,-106.18903,39.31512,-103.58598,LC08_L2SP_033032_20130318_20200913_02_T1,LC80330322013077LGN02
1,LC08_L2SP_033033_20130318_20200913_02_T1,LC08_L1TP_033033_20130318_20200913_02_T1,LC80330332013077LGN02,2013-03-18,T1,2,33,33,33,33,...,39.89577,-106.65514,39.90347,-104.02317,37.88251,-106.60905,37.88968,-104.05038,LC08_L2SP_033033_20130318_20200913_02_T1,LC80330332013077LGN02
2,LC08_L2SP_033032_20130323_20200912_02_T2,LC08_L1GT_033032_20130323_20200912_02_T2,LC80330322013082LGN02,2013-03-23,T2,2,33,32,33,32,...,41.33185,-105.08843,41.30317,-102.42949,39.33719,-105.08587,39.31046,-102.50399,LC08_L2SP_033032_20130323_20200912_02_T2,LC80330322013082LGN02


In [3]:
#  Grab only collects over the month of April, between 2023 and 2020
def filter(p, val):
    if val:
        mo = re.search(p,str(val))
        if mo:
            return True
        else:
            return False
    else:
        return False

regex_filter_date = lambda x: filter( pattern_date, x )
regex_filter_lc   = lambda x: filter( pattern_lc, x )
collects_to_request = collect_list.loc[collect_list['Landsat Product Identifier L2'].apply( regex_filter_lc ) &
                                       collect_list['Date Acquired'].apply( regex_filter_date )]
print( f'All Collections matching patterns' )
display(collects_to_request.head(collects_to_request.shape[0]))


All Collections matching patterns


,Landsat Product Identifier L2,Landsat Product Identifier L1,Landsat Scene Identifier,Date Acquired,Collection Category,Collection Number,WRS Path,WRS Row,Target WRS Path,Target WRS Row,...,Corner Upper Left Latitude,Corner Upper Left Longitude,Corner Upper Right Latitude,Corner Upper Right Longitude,Corner Lower Left Latitude,Corner Lower Left Longitude,Corner Lower Right Latitude,Corner Lower Right Longitude,Display ID,Entity ID
806,LC08_L2SP_032033_20200501_20200820_02_T1,LC08_L1TP_032033_20200501_20200820_02_T1,LC80320332020122LGN00,2020-05-01,T1,2,32,33,32,33,...,39.96050,-104.48600,39.91769,-101.80824,37.86271,-104.50092,37.82296,-101.90066,LC08_L2SP_032033_20200501_20200820_02_T1,LC80320332020122LGN00
807,LC08_L2SP_033032_20200508_20200820_02_T1,LC08_L1TP_033032_20200508_20200820_02_T1,LC80330322020129LGN00,2020-05-08,T1,2,33,32,33,32,...,41.38702,-105.60879,41.36841,-102.84298,39.27085,-105.59007,39.25358,-102.90927,LC08_L2SP_033032_20200508_20200820_02_T1,LC80330322020129LGN00
808,LC08_L2SP_033033_20200508_20200820_02_T1,LC08_L1TP_033033_20200508_20200820_02_T1,LC80330332020129LGN00,2020-05-08,T1,2,33,33,33,33,...,39.95432,-106.03492,39.94687,-103.32725,37.83515,-106.00461,37.82824,-103.37622,LC08_L2SP_033033_20200508_20200820_02_T1,LC80330332020129LGN00
809,LC08_L2SP_034032_20200515_20200820_02_T1,LC08_L1TP_034032_20200515_20200820_02_T1,LC80340322020136LGN00,2020-05-15,T1,2,34,32,34,32,...,41.37871,-107.18366,41.39781,-104.38871,39.24231,-107.11588,39.26003,-104.40770,LC08_L2SP_034032_20200515_20200820_02_T1,LC80340322020136LGN00
810,LC08_L2SP_034033_20200515_20200820_02_T1,LC08_L1TP_034033_20200515_20200820_02_T1,LC80340332020136LGN00,2020-05-15,T1,2,34,33,34,33,...,39.94844,-107.61152,39.97779,-104.87235,37.80946,-107.53433,37.83666,-104.87612,LC08_L2SP_034033_20200515_20200820_02_T1,LC80340332020136LGN00
811,LC08_L2SP_032033_20200517_20200820_02_T1,LC08_L1TP_032033_20200517_20200820_02_T1,LC80320332020138LGN00,2020-05-17,T1,2,32,33,32,33,...,39.96300,-104.44032,39.91912,-101.76255,37.86251,-104.45658,37.82179,-101.85641,LC08_L2SP_032033_20200517_20200820_02_T1,LC80320332020138LGN00
812,LC08_L2SP_033032_20200524_20200820_02_T1,LC08_L1TP_033032_20200524_20200820_02_T1,LC80330322020145LGN00,2020-05-24,T1,2,33,32,33,32,...,41.38713,-105.58727,41.36807,-102.82506,39.27096,-105.56920,39.25326,-102.89190,LC08_L2SP_033032_20200524_20200820_02_T1,LC80330322020145LGN00
813,LC08_L2SP_033033_20200524_20200820_02_T1,LC08_L1TP_033033_20200524_20200820_02_T1,LC80330332020145LGN00,2020-05-24,T1,2,33,33,33,33,...,39.95448,-106.01736,39.94656,-103.30618,37.83529,-105.98756,37.82795,-103.35578,LC08_L2SP_033033_20200524_20200820_02_T1,LC80330332020145LGN00
814,LC08_L2SP_034032_20200531_20200820_02_T1,LC08_L1TP_034032_20200531_20200820_02_T1,LC80340322020152LGN00,2020-05-31,T1,2,34,32,34,32,...,41.37898,-107.16932,41.39774,-104.37436,39.24256,-107.10199,39.25996,-104.39379,LC08_L2SP_034032_20200531_20200820_02_T1,LC80340322020152LGN00
815,LC08_L2SP_034033_20200531_20200820_02_T1,LC08_L1TP_034033_20200531_20200820_02_T1,LC80340332020152LGN00,2020-05-31,T1,2,34,33,34,33,...,39.94883,-107.59398,39.97777,-104.85829,37.80982,-107.51730,37.83665,-104.86249,LC08_L2SP_034033_20200531_20200820_02_T1,LC80340332020152LGN00


The API needs to be in a JSON format. 

In [4]:
request = { 'olitirs8_collection_2_l2': { 'inputs': list(collects_to_request['Landsat Product Identifier L2'].values[:]),
                                           'products': ['et'] },
             "format": "gtiff", 
             "note": "Datasets for April between 2023 and 2020."
           }
json_str = json.dumps( request )
json_str 

'{"olitirs8_collection_2_l2": {"inputs": ["LC08_L2SP_032033_20200501_20200820_02_T1", "LC08_L2SP_033032_20200508_20200820_02_T1", "LC08_L2SP_033033_20200508_20200820_02_T1", "LC08_L2SP_034032_20200515_20200820_02_T1", "LC08_L2SP_034033_20200515_20200820_02_T1", "LC08_L2SP_032033_20200517_20200820_02_T1", "LC08_L2SP_033032_20200524_20200820_02_T1", "LC08_L2SP_033033_20200524_20200820_02_T1", "LC08_L2SP_034032_20200531_20200820_02_T1", "LC08_L2SP_034033_20200531_20200820_02_T1", "LC08_L2SP_034032_20210502_20210508_02_T1", "LC08_L2SP_034033_20210502_20210508_02_T1", "LC08_L2SP_032033_20210504_20210517_02_T1", "LC08_L2SP_033032_20210511_20210524_02_T2", "LC08_L2SP_033033_20210511_20210524_02_T2", "LC08_L2SP_034032_20210518_20210528_02_T1", "LC08_L2SP_034033_20210518_20210528_02_T1", "LC08_L2SP_032033_20210520_20210528_02_T1", "LC08_L2SP_033032_20210527_20210607_02_T1", "LC08_L2SP_033033_20210527_20210607_02_T1", "LC08_L2SP_034032_20220505_20220512_02_T1", "LC08_L2SP_034033_20220505_2022051

In [5]:
curl_cmd = f"curl --netrc -d '{json_str}' https://espa.cr.usgs.gov/api/v0/order"
curl_cmd

'curl --netrc -d \'{"olitirs8_collection_2_l2": {"inputs": ["LC08_L2SP_032033_20200501_20200820_02_T1", "LC08_L2SP_033032_20200508_20200820_02_T1", "LC08_L2SP_033033_20200508_20200820_02_T1", "LC08_L2SP_034032_20200515_20200820_02_T1", "LC08_L2SP_034033_20200515_20200820_02_T1", "LC08_L2SP_032033_20200517_20200820_02_T1", "LC08_L2SP_033032_20200524_20200820_02_T1", "LC08_L2SP_033033_20200524_20200820_02_T1", "LC08_L2SP_034032_20200531_20200820_02_T1", "LC08_L2SP_034033_20200531_20200820_02_T1", "LC08_L2SP_034032_20210502_20210508_02_T1", "LC08_L2SP_034033_20210502_20210508_02_T1", "LC08_L2SP_032033_20210504_20210517_02_T1", "LC08_L2SP_033032_20210511_20210524_02_T2", "LC08_L2SP_033033_20210511_20210524_02_T2", "LC08_L2SP_034032_20210518_20210528_02_T1", "LC08_L2SP_034033_20210518_20210528_02_T1", "LC08_L2SP_032033_20210520_20210528_02_T1", "LC08_L2SP_033032_20210527_20210607_02_T1", "LC08_L2SP_033033_20210527_20210607_02_T1", "LC08_L2SP_034032_20220505_20220512_02_T1", "LC08_L2SP_03403

Using CURL: 

    $> curl --netrc -d '{"olitirs8_collection_2_l2": {"inputs": ["LC08_L2SP_033032_20200406_20200822_02_T1", "LC08_L2SP_033033_20200406_20200822_02_T1", "LC08_L2SP_034032_20200413_20200822_02_T1", "LC08_L2SP_034033_20200413_20200822_02_T1", "LC08_L2SP_032033_20200415_20200822_02_T1", "LC08_L2SP_033032_20200422_20200822_02_T1", "LC08_L2SP_033033_20200422_20200822_02_T1", "LC08_L2SP_034032_20200429_20200820_02_T1", "LC08_L2SP_034033_20200429_20200820_02_T1", "LC08_L2SP_032033_20210402_20210409_02_T1", "LC08_L2SP_033032_20210409_20210416_02_T1", "LC08_L2SP_033033_20210409_20210416_02_T1", "LC08_L2SP_034032_20210416_20210424_02_T1", "LC08_L2SP_034033_20210416_20210424_02_T1", "LC08_L2SP_032033_20210418_20210424_02_T1", "LC08_L2SP_033032_20210425_20210501_02_T1", "LC08_L2SP_033033_20210425_20210501_02_T1", "LC08_L2SP_034032_20220403_20220412_02_T1", "LC08_L2SP_034033_20220403_20220412_02_T1", "LC08_L2SP_032033_20220405_20220412_02_T1", "LC08_L2SP_033032_20220412_20220419_02_T1", "LC08_L2SP_033033_20220412_20220419_02_T1", "LC08_L2SP_034032_20220419_20220427_02_T1", "LC08_L2SP_034033_20220419_20220427_02_T1", "LC08_L2SP_032033_20220421_20220428_02_T1", "LC08_L2SP_033032_20220428_20220504_02_T1", "LC08_L2SP_033033_20220428_20220504_02_T1", "LC08_L2SP_034032_20230406_20230412_02_T1", "LC08_L2SP_034033_20230406_20230412_02_T1", "LC08_L2SP_032033_20230408_20230420_02_T1", "LC08_L2SP_033032_20230415_20230428_02_T1", "LC08_L2SP_033033_20230415_20230428_02_T1", "LC08_L2SP_032033_20230424_20230501_02_T1"], "products": ["et"]}, "format": "gtiff", "note": "Datasets for April between 2023 and 2020."}' https://espa.cr.usgs.gov/api/v0/order 

Response: 
    
    {"orderid":"espa-marvin_smith1@me.com-12262023-172441-821","status":"ordered"}

## Evapotranspiration Results

Baseline of Denver for May 27, 2021
![Downtown Denver](../../images/evp_20210521.png)

Observations:

* Sloan's Lake shows up bright red, which makes sense.
* The area around City Park is nice and bright, thanks to the golf course.
* The river is nice and bright.
* I'm happy to see downtown be nice and dark.